In [1]:
from z3 import Real, And, Or, Not, Solver, Exists, If
import z3
from typing import List
import pandas as pd
import numpy as np

In [2]:
def get_raw_value(expr: z3.ExprRef):
    try:
        if(isinstance(expr, z3.RatNumRef)):
            return expr.as_fraction()
        elif(isinstance(expr, z3.BoolRef)):
            return bool(expr)
        elif(isinstance(expr, z3.ArithRef)):
            return np.nan
        else:
            raise NotImplementedError
    except z3.z3types.Z3Exception as e:
        return np.nan


def get_model_value_list(model: z3.ModelRef, l: List[z3.ExprRef]):
    ret = []
    for var in l:
        val = get_raw_value(model.eval(var))
        ret.append(val)
    return ret

In [3]:
class Variables:
    def __init__(self, name: str, T: int):
        self.T = T
        self.A = [Real(f'{name}A_{t}') for t in range(T)]
        self.I = [Real(f'{name}I_{t}') for t in range(T)]
        self.S = [Real(f'{name}S_{t}') for t in range(T)]
        self.L = [Real(f'{name}L_{t}') for t in range(T)]
        self.C = Real(f'{name}C')
        self.buffer = Real(f'{name}buffer')
        
    def I_constr(self, s: Solver):
        C, A, I, S, L = self.C, self.A, self.I, self.S, self.L
        for t in range(1, self.T):
            s.add(I[t] <= A[t] - L[t])
            s.add(I[t] <= I[t-1] + C)
            s.add(Or(I[t] == A[t] - L[t], I[t] == I[t-1] + C))

        s.add(I[0] <= A[0] - L[0])
    
    def S_constr(self, s: Solver):
        C, buffer, A, I, S, L = self.C, self.buffer, self.A, self.I, self.S, self.L
        s.add(C > 0)
        s.add(buffer > 0)
        s.add(A[0] >= 0)
        s.add(S[0] >= 0)
        s.add(L[0] >= 0)

        for t in range(self.T):
            s.add(S[t] <= I[t])
            if t > 0:
                s.add(L[t] == If(A[t] - L[t-1] > I[t] + buffer, A[t] - I[t] - buffer, L[t-1]))
                s.add(S[t] >= I[t-1])
                s.add(A[t] >= A[t-1])
                s.add(S[t] >= S[t-1])
                s.add(L[t] >= L[t-1])
    
    def print_vars(self, m):
        C, A, I, S, L = self.C, self.A, self.I, self.S, self.L
        print("C", m[C], "buffer", m[self.buffer])
        #for t in range(self.T):
        #    if m[I[t]] is None:
        #        print(f'A={float(m[A[t]].as_fraction())}\tI= - \tS={float(m[S[t]].as_fraction())}')
        #    else:
        #        print(f'A={float(m[A[t]].as_fraction())}\tI={float(m[I[t]].as_fraction())}\tS={float(m[S[t]].as_fraction())}\tL={float(m[L[t]].as_fraction())}')
        #frame = []
        #frame['A'] = [float(m[x].as_fraction()) for x in A]
        frame = {}
        frame['A'] = get_model_value_list(m, A)
        frame['I'] = get_model_value_list(m, I)
        frame['S'] = get_model_value_list(m, S)
        frame['L'] = get_model_value_list(m, L)
        df = pd.DataFrame(frame).astype(float)
        print(df)

In [4]:
T = 3
s = Solver()

x = Variables('x', T)
x.I_constr(s)
x.S_constr(s)

y = Variables('y', T)
y.I_constr(s)
y.S_constr(s)

dummy_s = Solver()
z = Variables('z', T)
z.I_constr(dummy_s)
z.S_constr(dummy_s)

s.add(Not(Exists(z.I, And(*dummy_s.assertions()))))
#s.add(Not(And(*dummy_s.assertions())))
#z.I_constr(s)

for t in range(T):
    #s.add(z.A[t] == (x.A[t] + y.A[t]) / 2)
    #s.add(z.I[t] == (x.I[t] + y.I[t]) / 2)
    #s.add(z.S[t] == (x.S[t] + y.S[t]) / 2)
    s.add(And(x.A[t] == y.A[t], y.A[t] == z.A[t]))
    s.add(And(x.S[t] == y.S[t], y.S[t] == z.S[t]))
    s.add(And(x.L[t] == y.L[t], y.L[t] == z.L[t]))

s.add(z.C == (x.C + y.C) / 2)
s.add(z.buffer == (x.buffer + y.buffer) / 2)
    
if False:  
    s.add(Or(x.C != y.C, x.buffer != y.buffer))
    s.add(z.buffer == (x.buffer + y.buffer) / 2)
elif False:
    s.add(x.C == y.C)
    s.add(x.buffer != y.buffer)
else:
    s.add(x.C != y.C)
    s.add(x.buffer == y.buffer)

if False:
    s.add(x.C == 1)
    s.add(y.C == 3)
    s.add(x.A[0] == 0)
    s.add(x.buffer == 20)

sat = s.check()
print(sat)
if str(sat) == 'sat':
    m = s.model()
    x.print_vars(m)
    print("")
    y.print_vars(m)
    print("")
    z.print_vars(m)

unsat


In [124]:
41 - 19 - 37/2

3.5

In [126]:
41 - 19 - 20

2